In [28]:
from autogen import ConversableAgent
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import os

config_ollama_mistral = [
    {
        "model": "llama3-8b-8192",  # Loaded with LiteLLM command
        "api_key": os.environ["GROQ_API_KEY"],  # Not needed
        "base_url": "groq",  
    }
]

llm_mistral = {
    "seed": 25,
    "temperature": 0,
    "config_list": config_ollama_mistral,
    }

In [29]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

In [30]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi3:mini")

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="nomic-embed-text",
)


In [31]:
loaders = [ PyPDFLoader('/home/niel77/MechanicalAgents/data/Examples_small.pdf') ]
docs = []
for l in loaders:
    docs.extend(l.load())
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(docs)

In [32]:
from chromadb.utils import embedding_functions
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
                     documents=docs,                 # Data
                     embedding=embeddings,    # Embedding model
                     persist_directory="./chroma_db" # Directory to save data
                     )

In [33]:
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=embeddings   # Embedding model
                   )

In [34]:
retriever = vectorstore_disk.as_retriever()

In [35]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [36]:
qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore_disk.as_retriever(),
    memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
)

In [37]:
def cadquery_coder(design: str) -> str:
    system_prompt = (
    """You are a Cadquery code writer that retrieves the relevant code from the given context to create CAD models.
            Write your code script in Markdown format. For example:
            ```python
            import cadquery as cq
        from ocp_vscode import * # this is used to visualize model with OCP CAD viewer
        height = 60.0
        width = 80.0
        thickness = 10.0
        # make the base
        box = cq.Workplane("XY").box(height, width, thickness)
        show(box)
        # cq.exporters.export(box, "box.stl")
        # cq.exporters.export(box.section(), "box.dxf")
        # cq.exporters.export(box, "box.step")
        ##
        ```
    """
    "\n\n"
    "{context}"
        )
    vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=embeddings   # Embedding model
                   )
    retriever = vectorstore_disk.as_retriever()
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": design})
    return response['answer']


In [38]:
cadquery_coder("plate with hole")

'```python\nimport cadquery as cq\nfrom ocp_vscode import * # this is used to visualize model with OCP CAD viewer\n# Define the dimensions of the box. Feel free to modify these values directly in your CadQuery script instead if preferred for simplicity or efficiency reasons, but be aware that modifying object properties programmatically will require more complex code logic and careful management compared to using predefined shapes as above:\nlength = 80.0\nheight = 60.0\nthickness = 10.0\ncenter_hole_dia = 22.0\n# Create a box based on the dimensions provided, with specified thickness along all faces and then add center hole to it using \'Workplane()\' chain:\nresult = (\n    cq.Workplane("XY") # Start from XY plane for convenience in this example\n    .box(length, height, thickness)  # Generate a box of the specified dimensions\n    .faces(">Z")                        # Select top faces only to place hole on them later (for visualization purposes or if you prefer not including bottom 

In [39]:
# create an AssistantAgent instance named "assistant"
Cad_codewriter = ConversableAgent(
    "CAD Code Writer",
    system_message='''CAD Code Writer.You are a CadQuery expert and you write codes in Python to create CAD models using CADquery. 
        Here is an example of abox you created and saved in the step, dxf and stl format.
        ##
        Q: Create a box of size 80*60*10 and save it in stl, step and dxf file format.
        A:
        ```python
        import cadquery as cq
        from ocp_vscode import * # this is used to visualize model with OCP CAD viewer
        height = 60.0
        width = 80.0
        thickness = 10.0
        # make the base
        box = cq.Workplane("XY").box(height, width, thickness)
        show(box)
        # cq.exporters.export(box, "box.stl")
        # cq.exporters.export(box.section(), "box.dxf")
        # cq.exporters.export(box, "box.step")
        ##
        ```
        '''
        ,
    llm_config=llm_mistral,
    human_input_mode="NEVER",
)

code_critic = AssistantAgent(
    "CAD Code critic",
    system_message=''' You provide the correct code to the Cad Code Writer when there is error in
    his response by using the response from the function call. 
        ''',
    llm_config=llm_mistral,
    human_input_mode="NEVER",
)

# create a UserProxyAgent instance named "user_proxy"
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    code_execution_config= {
        "work_dir": "NewCAD1",
        "use_docker": False,
    },
    llm_config=llm_mistral,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
)

In [40]:

from typing import Annotated


# Register the function with the agent
@user_proxy.register_for_execution()
@Cad_codewriter.register_for_llm(description="Function to create CadQuery design code generator.")
def cadquery_design_generator(
    design: Annotated[str, "Desired CAD design"]
) -> str:
    response = cadquery_coder(design)
    return f"CadQuery code for the {design}:\n\n{response}"

In [41]:
user_proxy.function_map

{'cadquery_design_generator': <function __main__.cadquery_design_generator(design: Annotated[str, 'Desired CAD design']) -> str>}

In [42]:
result= user_proxy.initiate_chat(
    Cad_codewriter,
    message="""
Write the CadQuery code to create a plate with hole.
"""
)

user_proxy (to CAD Code Writer):


Write the CadQuery code to create a plate with hole.


--------------------------------------------------------------------------------


APIConnectionError: Connection error.

In [ ]:
print(result)

In [42]:
import autogen  
group_chat = autogen.GroupChat(agents=[user_proxy, Cad_codewriter, code_critic],speaker_selection_method= 'round_robin', messages=[], max_round=4)
manager = autogen.GroupChatManager(groupchat=group_chat, llm_config=llm_mistral)



In [ ]:
user_proxy.initiate_chat(
    manager,
    message="Write the CAdQuery Code to create a plate with hole.",
    silent = False,
    max_turns=10,
)